[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aicrumb/CQKP/blob/master/CQKP_Example.ipynb)

In [1]:
!pip install git+https://github.com/aicrumb/CQKP
from google.colab import output
output.clear() # we just do this so the notebook looks cleaner

In [2]:
import cqkp
model = cqkp.load_model()
output.clear()  # same here, just cleaning up

In [7]:
question = "what is a cow?"
answers = [
    "Cattle are large domesticated bovines. They are most widespread species of the genus Bos. ",
    "Cows eat grass and hay. On a dry matter basis a cow will eat 4 to 5 tons of forage per year."
]
print(model.best_answer(question, answers)[0])

Cattle are large domesticated bovines. They are most widespread species of the genus Bos. 


In [11]:
answer = "Cattle are large domesticated bovines. They are most widespread species of the genus Bos."
questions = [
    "what are pigs",
    "what are cows"
]
print(model.best_question(questions, answer)[0])

what are cows
